In [ ]:
from selenium import webdriver
driver = webdriver.Chrome('./driver/chromedriver')
driver.get('http://www.opinet.co.kr/searRgSelect.do')

In [ ]:
gu_list_raw = driver.find_element_by_xpath("""//*[@id="SIGUNGU_NM0"]""")
gu_list = gu_list_raw.find_elements_by_tag_name("option")
gu_names = [option.get_attribute('value') for option in gu_list]
gu_names.remove('')
gu_names

In [ ]:
element = driver.find_element_by_id('SIGUNGU_NM0')
element.send_keys(gu_names[0])

In [ ]:
xpath = """//*[@id="searRgSelect"]/span"""
element_sel_gu = driver.find_element_by_xpath(xpath).click()

In [ ]:
xpath = """//*[@id="glopopd_excel"]/span"""
element_get_excel = driver.find_element_by_xpath(xpath).click()

In [ ]:
import time
from tqdm import tqdm_notebook

# for gu in tqdm_notebook(gu_names):
#     element = driver.find_element_by_id("SIGUNGU_NM0")
#     element.send_keys(gu)
#     time.sleep(2)
    
#     xpath = """//*[@id="searRgSelect"]/span"""
#     element_sel_gu = driver.find_element_by_xpath(xpath).click()
#     time.sleep(1)
    
#     xpath = """//*[@id="glopopd_excel"]/span"""
#     element_get_excel = driver.find_element_by_xpath(xpath).click()
#     time.sleep(1)

In [ ]:
import pandas as pd
from glob import glob

In [ ]:
glob('../data/지역*.xls')

In [ ]:
stations_files = glob('../data/지역*.xls')
tmp_raw = []
for file_name in stations_files:
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)
station_raw = pd.concat(tmp_raw)

In [ ]:
station_raw.info()

In [ ]:
station_raw.head()

In [ ]:
stations = pd.DataFrame({'Oil_store': station_raw['상호'],
                         '주소': station_raw['주소'],
                         '가격': station_raw['휘발유'],
                         '셀프': station_raw['셀프여부'],
                         '상표': station_raw['상표'],
                        })

In [ ]:
stations.head()

In [ ]:
stations['구'] = [eachAddress.split()[1] for eachAddress in stations['주소']]

In [ ]:
stations.head()

In [ ]:
stations['구'].unique()

In [ ]:
stations[stations['구']=='서울특별시']

In [ ]:
stations.loc[stations['구']=='서울특별시', '구'] = '성동구'
stations[stations['Oil_store']=='SK네트웍스(주)효진주유소']

In [ ]:
stations['구'].unique()

In [ ]:
stations[stations['구']=='특별시']

In [ ]:
stations.loc[stations['구']=='특별시', '구'] = '도봉구'

In [ ]:
stations[stations['Oil_store']=='서현주유소']

In [ ]:
stations['구'].unique()

In [ ]:
stations[stations['가격']=='-']

In [ ]:
stations = stations[stations['가격'] != '-']

In [ ]:
stations.info()

In [ ]:
stations['가격'] = [float(value) for value in stations['가격']]

In [ ]:
stations.info()

In [ ]:
stations.reset_index(inplace=True)

In [ ]:
stations.head()

In [ ]:
del stations['index']

In [ ]:
stations.head()

In [ ]:
import platform
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = 'C:Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system')
%matplotlib inline

In [ ]:
stations.boxplot(column='가격', by='셀프', figsize=(12, 8))

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='상표', y='가격', hue='셀프', data=stations, palette='Set3')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='상표', y='가격', data=stations, palette='Set3')
sns.swarmplot(x='상표', y='가격', data=stations, color='.6')
plt.show()

In [ ]:
import json
import folium
import googlemaps
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
stations.sort_values(by='가격', ascending=False).head(10)

In [ ]:
stations.sort_values(by='가격', ascending=True).head(10)

In [ ]:
import numpy as np
gu_data = pd.pivot_table(stations, index=['구'], values=['가격'],
                         aggfunc=np.mean)
gu_data.head()

In [ ]:
geo_path = '../data/02.skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='UTF-8'))
map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data=geo_str,
               data=gu_data,
               columns=[gu_data.index, '가격'],
               fill_color='PuRd',
               key_on='feature.id')
map

In [ ]:
oil_price_top10 = stations.sort_values(by='가격', ascending=False).head(10)
oil_price_top10

In [ ]:
oil_price_bottom10 = stations.sort_values(by='가격', ascending=True).head(10)
oil_price_bottom10

In [ ]:
gmap_key = input('input your gmap key ')

In [ ]:
gmaps = googlemaps.Client(key=gmap_key)

In [ ]:
from tqdm import tqdm_notebook

lat = []
lng = []
for n in tqdm_notebook(oil_price_top10.index):
    try:
        tmp_add = str(oil_price_top10['주소'][n]).split('(')[0]
        tmp_map = gmaps.geocode(tmp_add)
        tmp_loc = tmp_map[0].get('geometry')
        lat.append(tmp_loc['location']['lat'])
        lng.append(tmp_loc['location']['lng'])
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print('Here is nan!')
        
oil_price_top10['lat'] = lat
oil_price_top10['lng'] = lng
oil_price_top10

In [ ]:
from tqdm import tqdm_notebook

lat = []
lng = []
for n in tqdm_notebook(oil_price_bottom10.index):
    try:
        tmp_add = str(oil_price_bottom10['주소'][n]).split('(')[0]
        tmp_map = gmaps.geocode(tmp_add)
        tmp_loc = tmp_map[0].get('geometry')
        lat.append(tmp_loc['location']['lat'])
        lng.append(tmp_loc['location']['lng'])
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print('Here is nan!')
        
oil_price_bottom10['lat'] = lat
oil_price_bottom10['lng'] = lng
oil_price_bottom10

In [ ]:
oil_price_top10.to_csv('../data/04.oil_price_top10.csv', sep=',', encoding='UTF-8')

In [ ]:
oil_price_bottom10.to_csv('../data/04.oil_price_bottom10.csv', sep=',', encoding='UTF-8')

In [ ]:
map = folium.Map(location=[37.5202, 126.975], zoom_start=10.5)
for n in oil_price_top10.index:
    if pd.notnull(oil_price_top10['lat'][n]):
        folium.CircleMarker([oil_price_top10['lat'][n],
                             oil_price_top10['lng'][n]],
                            radius=15, color='#CD3181',
                            fill_color='#CD3181',
                            fill=True).add_to(map)

for n in oil_price_bottom10.index:
    if pd.notnull(oil_price_bottom10['lat'][n]):
        folium.CircleMarker([oil_price_bottom10['lat'][n],
                             oil_price_bottom10['lng'][n]],
                             radius=15, color='#3186cc',
                             fill_color='#3186cc',
                             fill=True).add_to(map)
map